In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


# Importing libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
import optuna

# Load Datasets

In [3]:
train_df = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")
train_df_extra = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv")


# Combine Datasets

In [4]:
training_df = pd.concat([train_df, train_df_extra], ignore_index=True)

# Prepare Data

In [5]:
x = 500000
selected_rows = training_df.iloc[:x, :]
y = selected_rows["Price"]
X = selected_rows.drop(columns=["Price", "id"])

categorical_features = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
X[categorical_features] = X[categorical_features].fillna("None")
test_df[categorical_features] = test_df[categorical_features].fillna("None")

numeric_features = ["Compartments", "Weight Capacity (kg)"]
X[numeric_features] = X[numeric_features].fillna(X[numeric_features].mean())
test_df[numeric_features] = test_df[numeric_features].fillna(test_df[numeric_features].mean())

# Preprocessing

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Objective Function for Optuna

In [7]:
def objective(trial):
    model_type = trial.suggest_categorical("model_type", ["XGBoost", "CatBoost"])

    if model_type == "XGBoost":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10)
        }
        model = XGBRegressor(**params)

    else:  
        params = {
            "iterations": trial.suggest_int("iterations", 100, 1000, step=50),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
            "depth": trial.suggest_int("depth", 3, 10),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
            "random_strength": trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
        }
        model = CatBoostRegressor(**params, verbose=0)

    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("regressor", model)
    ])

    rmse_scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
    return -rmse_scores.mean()  

# Run Optuna to optimize the model

In [8]:
study = optuna.create_study(direction="minimize")  
study.optimize(objective, n_trials=50) 


print("Best Hyperparameters:", study.best_params)

[I 2025-02-27 17:49:16,024] A new study created in memory with name: no-name-e8e7c2f3-e33b-46c0-b205-4b7cf6fce34d
[I 2025-02-27 17:49:31,971] Trial 0 finished with value: 38.94788660725072 and parameters: {'model_type': 'XGBoost', 'n_estimators': 200, 'learning_rate': 0.01741317840180417, 'max_depth': 6, 'subsample': 0.9631240798189427, 'colsample_bytree': 0.6709908044916231, 'reg_lambda': 0.0032528142326923412, 'reg_alpha': 0.04260306838472462, 'min_child_weight': 8}. Best is trial 0 with value: 38.94788660725072.
[I 2025-02-27 17:50:13,445] Trial 1 finished with value: 38.95534440606983 and parameters: {'model_type': 'CatBoost', 'iterations': 450, 'learning_rate': 0.010462397041003189, 'depth': 7, 'l2_leaf_reg': 0.0047004514154657655, 'random_strength': 7.947772542386924}. Best is trial 0 with value: 38.94788660725072.
[I 2025-02-27 17:50:25,852] Trial 2 finished with value: 38.97349075744159 and parameters: {'model_type': 'XGBoost', 'n_estimators': 100, 'learning_rate': 0.0557838841

Best Hyperparameters: {'model_type': 'XGBoost', 'n_estimators': 700, 'learning_rate': 0.014866156138676145, 'max_depth': 4, 'subsample': 0.8468415621733095, 'colsample_bytree': 0.8955732311163208, 'reg_lambda': 0.00927207627481951, 'reg_alpha': 3.900209627613867, 'min_child_weight': 6}


# Train Final Model

In [9]:
best_params = study.best_params
final_model_type = best_params.pop("model_type")


models = []
if final_model_type == "XGBoost":
    models.append(XGBRegressor(**best_params))
else:  # CatBoost
    models.append(CatBoostRegressor(**best_params, verbose=0))


models.append(XGBRegressor(n_estimators=100, learning_rate=0.1))
models.append(CatBoostRegressor(iterations=100, learning_rate=0.1, verbose=0))


voting_model = VotingRegressor(estimators=[(f'model_{i}', model) for i, model in enumerate(models)])
voting_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", voting_model)
])

# Train Voting Regressor

In [10]:
voting_pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Compartments',
                                                   'Weight Capacity (kg)']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Brand', 'Material', 'Size',
                                                   'Laptop Compartment',
                                                   'Waterproof', 'Style',
                                                   'Color'])])),
                ('regressor',
                 VotingRegressor(estimators=[('model_0',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           ca...
                                                           learning_rate=0.1,
                                                           max_bin=None,
                                                           max_cat_threshold=None,
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=None,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           multi_strategy=None,
                                                           n_estimators=100,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=None, ...)),
                                             ('model_2',
                                              <catboost.core.CatBoostRegressor object at 0x7d296e31b070>)]))])

# Make Predictions

In [11]:
final_preds = voting_pipeline.predict(X_test)
final_rmse = mean_squared_error(y_test, final_preds, squared=False)
print(f"Final RMSE from Voting Model: {final_rmse}")

Final RMSE from Voting Model: 38.97458003390934


# Prepare Submission

In [12]:
X_submission = test_df[X.columns]
y_pred_submission = voting_pipeline.predict(X_submission)

submission = pd.DataFrame({'id': test_df['id'].values, 'Price': y_pred_submission})
submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'")

Submission file saved as 'submission.csv'
